<a href="https://colab.research.google.com/github/tooruui39/marketing-project/blob/main/%E5%92%96%E5%95%A1%E5%BB%B3%E8%A1%8C%E9%8A%B7%E5%88%86%E6%9E%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive 

drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
myfolder = "drive/My Drive/Learn-Statistics/DATA/"

In [6]:
import pandas as pd
from sklearn import linear_model 
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from statsmodels.stats.outliers_influence import variance_inflation_factor
import copy
import math
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
cafe = pd.read_csv(myfolder + "CAFE.csv")
cafe.head()

,CAFE_ID,STATION_USER,MINUTES_FROM_STATION,COMPETITOR,AREA,SHEETS,LEADER_AGE,WORKERS,MENU,ADS,SALES,SALES_RANK
0,CAFE0001,58690,2,3,29,16,30,14,19,264803,2956215,E
1,CAFE0002,74627,0,4,33,17,33,14,21,272278,2686770,E
2,CAFE0003,94738,0,4,54,27,29,22,23,265086,4094574,C
3,CAFE0004,97464,0,3,59,31,27,19,23,347455,4754772,B
4,CAFE0005,81798,2,3,50,25,29,16,21,289238,4393329,C


In [8]:
columns = ["STATION_USER","MINUTES_FROM_STATION","COMPETITOR","AREA","SHEETS","LEADER_AGE","WORKERS","MENU","ADS"]
X_var = cafe.loc[:,columns]
y_var = cafe.loc[:,["SALES"]]

X_train, X_test, y_train, y_test = train_test_split(X_var, y_var, test_size=0.5, random_state=42)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(64, 9)
(64, 9)
(64, 1)
(64, 1)


In [9]:
std_object_x = StandardScaler()
std_object_x.fit(X_train)
std_X_train = pd.DataFrame(std_object_x.transform(X_train),columns=columns)
std_X_test = pd.DataFrame(std_object_x.transform(X_test),columns=columns)

In [10]:
std_object_y = StandardScaler()
std_object_y.fit(y_train)
std_y_train = pd.DataFrame(std_object_y.transform(y_train),columns=["SALES"])
std_y_test = pd.DataFrame(std_object_y.transform(y_test),columns=["SALES"])

In [11]:
reg = linear_model.LinearRegression()
X = X_train
y = y_train

reg.fit(X,y)
reg.intercept_


array([3156651.11337209])

In [12]:
reg.coef_

array([[ 1.88985898e+01,  2.87269415e+05, -1.68578691e+05,
         2.22989235e+04,  1.09493160e+04, -5.77848978e+04,
         5.20795203e+02,  1.31572525e+04, -4.37478692e-01]])

In [13]:
reg.score(X,y)

0.6136827747551008

In [14]:
def calcAIC(x, y_data, y_predict):
  colNum = len(x.columns)
  rowNum = x.count()[0]
  y_data = y_data.values
  RSS = ( ( y_data - y_predict ) * ( y_data - y_predict ) ).sum()
  AIC = rowNum * ( math.log( 2 * math.pi * RSS / rowNum ) + 1 ) + 2 * ( colNum + 2 )
  return AIC

In [15]:
def LR_stepwise_AIC(X, y, maxIter=100):
  reg = linear_model.LinearRegression()
  returnReg = linear_model.LinearRegression()
  orgColList = X.columns
  modelColList = []
  maxR2modelColList = []
  modelCoef = 0
  modelIntercept = 0
  modelR2 = 0
  modelAIC = 1000000000
  residueColList = copy.copy(orgColList)
  # 
  count = 0
  while count < maxIter:
    count = count + 1
    for i in range(len(residueColList)):
      tmpColList = copy.copy(modelColList)
      if residueColList[i] in tmpColList:
        if len(tmpColList) > 1:
          tmpColList.remove( residueColList[i] )
        else:
          continue
      else:
        tmpColList.append( residueColList[i] )
      tmp_X = X.loc[:,tmpColList]
      reg.fit(tmp_X, y)
      if modelAIC > calcAIC(tmp_X, y, reg.predict(tmp_X) ):
        maxR2modelColList = copy.copy(tmpColList)
        modelR2 = reg.score(tmp_X, y)
        modelAIC = calcAIC(tmp_X, y, reg.predict(tmp_X) )
        print("model update: variables:" + str(maxR2modelColList) + ", AIC: " + str(modelAIC) )
        returnReg = copy.copy(reg)
    modelColList = copy.copy(maxR2modelColList)
  return returnReg, modelColList

In [16]:
reg, columns = LR_stepwise_AIC(std_X_train, std_y_train, maxIter=10)

model update: variables:['STATION_USER'], AIC: 177.8965101003825
model update: variables:['MINUTES_FROM_STATION'], AIC: 171.74825534980306
model update: variables:['AREA'], AIC: 167.40337197867618
model update: variables:['AREA', 'STATION_USER'], AIC: 150.36811213618202
model update: variables:['AREA', 'STATION_USER', 'MINUTES_FROM_STATION'], AIC: 138.5121613237411
model update: variables:['AREA', 'STATION_USER', 'MINUTES_FROM_STATION', 'COMPETITOR'], AIC: 136.46067654427048
model update: variables:['AREA', 'STATION_USER', 'MINUTES_FROM_STATION', 'COMPETITOR', 'LEADER_AGE'], AIC: 135.36303182956624


結果分析：
透過stepwise法(AIC)從九個變數裡挑出與目標變數（銷售額）相關性較高的五個變出投入訓練模型

In [17]:
columns_aic = ['AREA', 'STATION_USER', 'MINUTES_FROM_STATION', 'COMPETITOR', 'LEADER_AGE']
X_var = cafe.loc[:,columns_aic]
y_var = cafe.loc[:,["SALES"]]

X_train, X_test, y_train, y_test = train_test_split(X_var, y_var, test_size=0.5, random_state=42)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(64, 5)
(64, 5)
(64, 1)
(64, 1)


In [19]:
std_object_x = StandardScaler()
std_object_x.fit(X_train)
std_X_train = pd.DataFrame(std_object_x.transform(X_train),columns=columns)
std_X_test = pd.DataFrame(std_object_x.transform(X_test),columns=columns)

In [20]:
std_object_y = StandardScaler()
std_object_y.fit(y_train)
std_y_train = pd.DataFrame(std_object_y.transform(y_train),columns=["SALES"])
std_y_test = pd.DataFrame(std_object_y.transform(y_test),columns=["SALES"])

In [18]:
from sklearn.linear_model import Lasso

In [21]:
reg_lasso = Lasso(alpha=0.1)
reg_lasso.fit(std_X_train,std_y_train)

Lasso(alpha=0.1)

In [28]:
def checkVIF( ExplanatoryVarDataSet ):
  tmp_columnList = ExplanatoryVarDataSet.columns
  vifList = []
  for i in range(len(tmp_columnList)):
    colname = tmp_columnList[i]
    vif = variance_inflation_factor(ExplanatoryVarDataSet.values, i)
    vifList.append( [ colname, vif ] )
  return  pd.DataFrame( vifList, columns=["COLUMN","VIF"] )

In [29]:
#再一次檢查多重共線性

result = pd.DataFrame( [ X_var.columns + ["intercept"]  , reg_lasso.coef_.tolist() + [reg_lasso.intercept_[0]] ] ).T
result.columns = ["COLUMN","COEF"]
vif = checkVIF(std_X_train[X_var.columns])
pd.merge( result, vif, on="COLUMN", how="outer")

,COLUMN,COEF,VIF
0,AREAintercept,0.356717,NaN
1,STATION_USERintercept,0.333728,NaN
2,MINUTES_FROM_STATIONintercept,0.272039,NaN
3,COMPETITORintercept,-0.067987,NaN
4,LEADER_AGEintercept,-0.018758,NaN
5,NaN,0.0,NaN
6,AREA,NaN,1.161750
7,STATION_USER,NaN,1.018544
8,MINUTES_FROM_STATION,NaN,1.153095
9,COMPETITOR,NaN,1.046378


In [31]:
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, median_absolute_error
import numpy as np

In [25]:
lasso_list = []
lasso_list.append(r2_score(std_y_test, reg_lasso.predict(std_X_test)))
lasso_list.append(mean_squared_error(std_y_test, reg_lasso.predict(std_X_test)))
lasso_list.append(mean_absolute_error(std_y_test, reg_lasso.predict(std_X_test)))
lasso_list.append(median_absolute_error(std_y_test, reg_lasso.predict(std_X_test)))

print(lasso_list)

[0.7640238585395739, 0.2572740788736443, 0.4162042940353184, 0.39218632606720727]


In [32]:
comparison = pd.DataFrame(np.array([lasso_list]), columns=['r2_score','mean_squared_error', 'mean_absolute_error', 'median_absolute_error']
                          ,index=['lasso'])
comparison

,r2_score,mean_squared_error,mean_absolute_error,median_absolute_error
lasso,0.764024,0.257274,0.416204,0.392186
